In [46]:
import openmm, sys, mdtraj
sys.path.append("../../")
from enhanced_sampling import utils, system_building as sb, system_saving as ss

In [84]:
openmm.version.full_version

'7.7.0.dev-130124a'

In [47]:
input_dict = sb.load_input_dir("../systems/system00", "../forcefield_data/charmm-params/")

Loading psf from ../systems/system00/step5_input.psf
Loading Charmm params from ../forcefield_data/charmm-params/
Loading positions from ../systems/system00/final_frame.cif
Loading box vectors from ../systems/system00/sysinfo.dat
Setting box size x: 121.162246 A, y: 121.162246 A, z:110.56 A


In [48]:
psf = input_dict["psf"]

In [49]:
psf.topology

<Topology; 5 chains, 31576 residues, 151920 atoms, 121416 bonds>

In [50]:
cif = input_dict['cif']

In [51]:
cif.topology

<Topology; 5 chains, 31576 residues, 151920 atoms, 121396 bonds>

In [52]:
input_pdb = openmm.app.PDBFile("/Users/alexpayne/Scientific_Projects/tmem175-allostery-analysis/complete-open-models/20211122-sys06-charmm-gui-extended/openmm/step5_input.pdb")

In [53]:
input_pdb.topology

<Topology; 5 chains, 31576 residues, 151920 atoms, 74116 bonds>

In [54]:
params = input_dict['params']

In [55]:
params

In [56]:
param_dict_with_units = sb.load_simulation_params("../sim_params/local_test.yaml")

In [57]:
param_dict_with_units

{'hydrogen_mass': Quantity(value=4.0, unit=dalton),
 'temperature': Quantity(value=303.15, unit=kelvin),
 'friction': Quantity(value=1, unit=/picosecond),
 'time_step': Quantity(value=0.004, unit=picosecond),
 'pressure': Quantity(value=1, unit=bar),
 'surface_tension': 0,
 'nonbonded_method': PME,
 'constraints': HBonds,
 'nsteps': 500000,
 'report_freq': 50000,
 'chk_freq': 250000,
 'traj_freq': 500000,
 'virtual_bond': ['resSeq 46 and name CA and chainid 0',
  'resSeq 46 and name CA and chainid 1'],
 'platform': 'CPU',
 'precision': 'mixed'}

In [58]:
type(param_dict_with_units['nonbonded_method'])

openmm.app.forcefield.PME

In [59]:
type(param_dict_with_units['constraints'])

openmm.app.forcefield.HBonds

In [60]:
system = psf.createSystem(input_dict["params"],
                              nonbondedMethod=param_dict_with_units['nonbonded_method'],
                              constraints=param_dict_with_units['constraints'],
                              removeCMMotion=False,
                              hydrogenMass=param_dict_with_units['hydrogen_mass'])

In [61]:
# integrator = openmm.LangevinMiddleIntegrator(param_dict_with_units['temperature'],
#                                              param_dict_with_units['friction'],
#                                              param_dict_with_units['time_step'])

In [62]:
integrator = openmm.LangevinIntegrator(param_dict_with_units['temperature'],
                                             param_dict_with_units['friction'],
                                             param_dict_with_units['time_step'])

In [92]:
barostat = openmm.MonteCarloMembraneBarostat(param_dict_with_units['pressure'],
                                         param_dict_with_units['surface_tension'],
                                         param_dict_with_units['temperature'],
                                         openmm.MonteCarloMembraneBarostat.XYIsotropic,
                                         openmm.MonteCarloMembraneBarostat.ZFree
                                         )
barostat.setFrequency(50)  ## for some reason the __init__ won't accept it as an argument, but this works
    ## the default is 25 timesteps, i've set it for 50
system.addForce(barostat)

9

8

In [65]:
platform = sb.get_platform_from_params(param_dict_with_units)

In [66]:
platform.getName()

'CPU'

In [67]:
sim = openmm.app.Simulation(psf.topology,
                                system=system,
                                integrator=integrator,
                                platform=platform)

In [74]:
# sim.context.setPositions(cif.positions)
sim.context.setPositions(input_pdb.positions)
sim.context.setVelocitiesToTemperature(param_dict_with_units['temperature'])

In [79]:
_state = sim.context.getState(getVelocities=True)

In [80]:
_state.getPeriodicBoxVectors()

Quantity(value=[Vec3(x=12.1162246, y=0.0, z=0.0), Vec3(x=0.0, y=12.1162246, z=0.0), Vec3(x=0.0, y=0.0, z=11.056000000000001)], unit=nanometer)

In [71]:
## Run minimization
print(
    "  initial : %8.3f kcal/mol"
    % (
        sim.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)

  initial : 273102.865 kcal/mol


In [72]:
sim.minimizeEnergy()
print(
    "  final : %8.3f kcal/mol"
    % (
        sim.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)

OpenMMException: Particle coordinate is nan

# test loading state

In [87]:
import bz2
input_state_path = "../systems/system00/state.xml.bz2"
with bz2.open(input_state_path, 'rb') as infile:
    state = openmm.XmlSerializer.deserialize(infile.read().decode())
sim.context.setState(state)

In [88]:
## Run minimization
print(
    "  initial : %8.3f kcal/mol"
    % (
        sim.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)

  initial : -323575.657 kcal/mol


In [89]:
sim.step(10)

In [90]:
state.getPeriodicBoxVectors()

Quantity(value=[Vec3(x=11.57873269797354, y=0.0, z=0.0), Vec3(x=0.0, y=11.57873269797354, z=0.0), Vec3(x=0.0, y=0.0, z=10.983541137965132)], unit=nanometer)

# I bet its because of the box vectors

In [95]:
x,y,z = state.getPeriodicBoxVectors()

In [97]:
print(x,y,z)

Vec3(x=11.57873269797354, y=0.0, z=0.0) nm Vec3(x=0.0, y=11.57873269797354, z=0.0) nm Vec3(x=0.0, y=0.0, z=10.983541137965132) nm


In [101]:
psf.setBox(x[0],y[1],z[2])

In [105]:
psf = input_dict["psf"]
system2 = psf.createSystem(input_dict["params"],
                              nonbondedMethod=param_dict_with_units['nonbonded_method'],
                              constraints=param_dict_with_units['constraints'],
                              removeCMMotion=False,
                              hydrogenMass=param_dict_with_units['hydrogen_mass'])
integrator2 = openmm.LangevinIntegrator(param_dict_with_units['temperature'],
                                             param_dict_with_units['friction'],
                                             param_dict_with_units['time_step'])
barostat2 = openmm.MonteCarloMembraneBarostat(param_dict_with_units['pressure'],
                                         param_dict_with_units['surface_tension'],
                                         param_dict_with_units['temperature'],
                                         openmm.MonteCarloMembraneBarostat.XYIsotropic,
                                         openmm.MonteCarloMembraneBarostat.ZFree
                                         )
barostat2.setFrequency(50)  ## for some reason the __init__ won't accept it as an argument, but this works
    ## the default is 25 timesteps, i've set it for 50
system2.addForce(barostat2)
platform2 = sb.get_platform_from_params(param_dict_with_units)

In [106]:
sim = openmm.app.Simulation(psf.topology,
                                system=system2,
                                integrator=integrator2,
                                platform=platform2)

In [107]:
sim.context.setPositions(cif.positions)
sim.context.setVelocitiesToTemperature(param_dict_with_units['temperature'])

### i was correct

In [108]:
print(
    "  initial : %8.3f kcal/mol"
    % (
        sim.context.getState(getEnergy=True).getPotentialEnergy()
        / openmm.unit.kilocalories_per_mole
    )
)

  initial : -323575.403 kcal/mol


In [109]:
psf2 = openmm.app.CharmmPsfFile("../systems/system00/step5_input.psf")